In [ ]:
# Import necessary packages and download NLTK resources
import os
import re
import io
import time
import nltk
import torch
import numpy as np
import torch.nn as nn
from os.path import join
from nltk import pos_tag
import torch.nn.functional as F
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix
from scipy.stats import randint as sp_randint
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

'''nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')'''


In [ ]:
#Path to data directory
data_dir = join(os.getcwd(), "Data")

# Define data sets
training_set = 'twitter-training-data.txt'
testsets = ['twitter-test1.txt', 'twitter-test2.txt', 'twitter-test3.txt']

In [ ]:
# Skeleton: Evaluation code for the test sets
def read_test(testset):
    '''
    readin the testset and return a dictionary
    :param testset: str, the file name of the testset to compare
    '''
    id_gts = {}
    with open(testset, 'r', encoding='utf8') as fh:
        for line in fh:
            fields = line.split('\t')
            tweetid = fields[0]
            gt = fields[1]

            id_gts[tweetid] = gt

    return id_gts


def confusion(id_preds, testset, classifier):
    '''
    print the confusion matrix of {'positive', 'negative'} between preds and testset
    :param id_preds: a dictionary of predictions formated as {<tweetid>:<sentiment>, ... }
    :param testset: str, the file name of the testset to compare
    :classifier: str, the name of the classifier
    '''
    id_gts = read_test(testset)

    gts = []
    for m, c1 in id_gts.items():
        if c1 not in gts:
            gts.append(c1)

    gts = ['positive', 'negative', 'neutral']

    conf = {}
    for c1 in gts:
        conf[c1] = {}
        for c2 in gts:
            conf[c1][c2] = 0

    for tweetid, gt in id_gts.items():
        if tweetid in id_preds:
            pred = id_preds[tweetid]
        else:
            pred = 'neutral'
        conf[pred][gt] += 1

    print(''.ljust(12) + '  '.join(gts))

    for c1 in gts:
        print(c1.ljust(12), end='')
        for c2 in gts:
            if sum(conf[c1].values()) > 0:
                print('%.3f     ' % (conf[c1][c2] / float(sum(conf[c1].values()))), end='')
            else:
                print('0.000     ', end='')
        print('')

    print('')


def evaluate(id_preds, testset, classifier):
    '''
    print the macro-F1 score of {'positive', 'negative'} between preds and testset
    :param id_preds: a dictionary of predictions formated as {<tweetid>:<sentiment>, ... }
    :param testset: str, the file name of the testset to compare
    :classifier: str, the name of the classifier
    '''
    id_gts = read_test(testset)

    acc_by_class = {}
    for gt in ['positive', 'negative', 'neutral']:
        acc_by_class[gt] = {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0}

    catf1s = {}

    ok = 0
    for tweetid, gt in id_gts.items():
        if tweetid in id_preds:
            pred = id_preds[tweetid]
        else:
            pred = 'neutral'

        if gt == pred:
            ok += 1
            acc_by_class[gt]['tp'] += 1
        else:
            acc_by_class[gt]['fn'] += 1
            acc_by_class[pred]['fp'] += 1

    catcount = 0
    itemcount = 0
    macro = {'p': 0, 'r': 0, 'f1': 0}
    micro = {'p': 0, 'r': 0, 'f1': 0}
    semevalmacro = {'p': 0, 'r': 0, 'f1': 0}

    microtp = 0
    microfp = 0
    microtn = 0
    microfn = 0
    for cat, acc in acc_by_class.items():
        catcount += 1

        microtp += acc['tp']
        microfp += acc['fp']
        microtn += acc['tn']
        microfn += acc['fn']

        p = 0
        if (acc['tp'] + acc['fp']) > 0:
            p = float(acc['tp']) / (acc['tp'] + acc['fp'])

        r = 0
        if (acc['tp'] + acc['fn']) > 0:
            r = float(acc['tp']) / (acc['tp'] + acc['fn'])

        f1 = 0
        if (p + r) > 0:
            f1 = 2 * p * r / (p + r)

        catf1s[cat] = f1

        n = acc['tp'] + acc['fn']

        macro['p'] += p
        macro['r'] += r
        macro['f1'] += f1

        if cat in ['positive', 'negative']:
            semevalmacro['p'] += p
            semevalmacro['r'] += r
            semevalmacro['f1'] += f1

        itemcount += n

    micro['p'] = float(microtp) / float(microtp + microfp)
    micro['r'] = float(microtp) / float(microtp + microfn)
    micro['f1'] = 2 * float(micro['p']) * micro['r'] / float(micro['p'] + micro['r'])

    semevalmacrof1 = semevalmacro['f1'] / 2

    print(testset + ' (' + classifier + '): %.3f' % semevalmacrof1)
    return '%.3f' % semevalmacrof1

In [ ]:
#function to read in training set, dev set, and testing sets
def read_in_file(data_dir, filename):
    data = []
    with io.open(join(data_dir, filename), 'r', encoding='utf8') as f:
        for line in f:
            line = line.lower()
            line = line.strip()
            data.append(line)
    return data
        
def preprocessing(data):
    #remove the first two sections of each line in data, keeping just the last column
    tweet_content = [line.split("\t")[-1] for line in data]

    #use regex to remove all urls
    def remove_urls(text):
        url_regex = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        return url_regex.sub('', text)

    #use regex to remove all words that start with @
    def remove_at(text):
        at_regex = re.compile(r'@[a-zA-Z0-9_]+')
        return at_regex.sub('', text)

    #use regex to remove all special characters
    def remove_special_characters(text):
        special_characters_regex = re.compile(r'[^a-zA-Z0-9 ]')
        return special_characters_regex.sub('', text)

    #use regex to remove numbers that are not part of a word
    def remove_numbers(text):
        numbers_regex = re.compile(r'(?<![a-zA-Z0-9_])\d+(?![a-zA-Z0-9_])')
        return numbers_regex.sub('', text)
    
    #use regex to remove stopwords
    def remove_stopwords(text):
        stopwords_list = stopwords.words('english')
        return' '.join([word for word in text.split() if word not in stopwords_list])

    #run the above functions on each line of data
    tweet_content = [remove_stopwords(remove_numbers(remove_special_characters(remove_at(remove_urls(line))))) for line in tweet_content]

    #replace the last item in each line of data with the same line in tweet_content
    for i, line in enumerate(data):
        data[i] = line.replace(line.split("\t")[-1], tweet_content[i]) 
    
    return data

In [ ]:
# Load training set, dev set and testing set
data = {}
tweetids = {}
tweetgts = {}
tweets = {}

for dataset in ['twitter-training-data.txt'] + testsets:
    data[dataset] = [preprocessing(read_in_file(data_dir, dataset))]
    tweets[dataset] = [line.split("\t")[2] for line in preprocessing(read_in_file(data_dir, dataset))]
    tweetids[dataset] = [line.split("\t")[0] for line in preprocessing(read_in_file(data_dir, dataset))]
    tweetgts[dataset] = [line.split("\t")[1] for line in preprocessing(read_in_file(data_dir, dataset))]


In [ ]:
#tokenize each line of tweets
for dataset in ['twitter-training-data.txt'] + testsets:
    try:
        tweets[dataset] = [word_tokenize(line) for line in tweets[dataset]]
    except:
        tweets[dataset] = [word_tokenize(line) for line in [line.split("\t")[2] for line in preprocessing(read_in_file(data_dir, dataset))]]

In [ ]:
#function to find the part of speech tagging in wordnet format for words in tweets
def pos_tagging(word):
    tag = nltk.pos_tag([word])[0][1]
    if tag.startswith("J"):
        return "a"
    elif tag.startswith("V"):
        return "v"
    elif tag.startswith("N"):
        return "n"
    elif tag.startswith("R"):
        return "r"
    else:
        return ""

#lemmatize the words in tweets
lemmatizer = WordNetLemmatizer()
for dataset in ['twitter-training-data.txt'] + testsets:
    for i, tweet in enumerate(tweets[dataset]):
        for j, word in enumerate(tweet):
            if pos_tagging(word) != "":
                tweets[dataset][i][j] = lemmatizer.lemmatize(word, pos=pos_tagging(word))



In [ ]:
#function to remove the file extension from a .txt file
def remove_extension(filename):
    return filename.split(".")[0]

In [16]:
print(', '.join(tweets[training_set][0]))

felt, privileged, play, foo, fighter, song, guitar, today, one, plectrum, gig, saturday


In [ ]:
#setting training set and test sets
training_set = 'twitter-training-data.txt'
testset = 'twitter-test1.txt'
testset_path = data_dir + '\\' + testset

In [56]:
#feature generation function
class TweetsDataset(Dataset):
    def __init__(self, tweets, tweetgts, word_vectors, max_tweet_length):
        self.tweets = tweets
        self.labels = tweetgts
        self.word_vectors = word_vectors
        self.max_tweet_length = max_tweet_length
        
    def __len__(self):
        return len(self.tweets)
    
    def __getitem__(self, idx):
        tweet = self.tweets[idx].split(',')
        tweet_vector = []
        for word in tweet:
            word = ''.join(filter(str.isalnum, word))
            if word in self.word_vectors:
                tweet_vector.append(self.word_vectors[word])
        tweet_vector = np.asarray(tweet_vector)
        if len(tweet_vector) > 0:
            tweet_vector = np.pad(tweet_vector, 
                    ((0, self.max_tweet_length - len(tweet_vector)), (0,0)), 
                    'constant', constant_values=0)
        else: 
            tweet_vector = np.zeros((self.max_tweet_length, 100))
        label = self.labels[idx]
        if label == 'positive':
            label = 0.0
        elif label == 'negative':
            label = 1.0
        else:
            label = 2.0
        return tweet_vector.astype(np.float32), label
    
def generate_features(training_set, testset, feature: str):
    training_tweets = [', '.join(tweet) for tweet in tweets[training_set]]
    test_tweets = [', '.join(tweet) for tweet in tweets[testset]]
    if feature == 'bow':
        #create feature vector using tfidf for bag of words
        vectorizer = TfidfVectorizer(min_df = 5, max_df = 0.8, sublinear_tf = True, use_idf = True)
        train_vectors = vectorizer.fit_transform(training_tweets)
        test_vectors = vectorizer.transform(test_tweets)

    elif feature == 'embeddings':
        #word embedding using glove
        glove_path = data_dir + '\\glove.6B\\' + 'glove.6B.100d.txt'
        word_vectors = {}

        with open(glove_path, 'r', encoding='utf-8') as f:
            for line in f:
                word = line.split()[0]
                vector = np.asarray(line.split()[1:])
                word_vectors[word] = vector
                
        #finding the maximum tweet length for padding
        longest_training_length = 0
        for tweet in training_tweets:
            tweet = tweet.split(',')
            length = len(tweet)
            if length > longest_training_length:
                longest_training_length = length
        
        longest_test_length = 0
        for tweet in test_tweets:
            tweet = tweet.split(',')
            length = len(tweet)
            if length > longest_test_length:
                longest_test_length = length
        
        batch_size = 128
        train_dataset = TweetsDataset(training_tweets, tweetgts[training_set], word_vectors, longest_training_length)
        train_vectors = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        test_dataset = TweetsDataset(test_tweets, tweetgts[testset], word_vectors, longest_test_length)
        test_vectors = DataLoader(test_dataset, batch_size=batch_size)
            
    return train_vectors, test_vectors



In [ ]:
#perform svm classification with a linear kernel
train_vectors, test_vectors = generate_features(training_set, testset, features)
svm = LinearSVC()
svm.fit(train_vectors, tweetgts[training_set])
predictions = svm.predict(test_vectors)

#testing on the testsets
idpreds = {}
testset_path = data_dir + '\\' + testset
for i, key in enumerate(tweetids[testset]):
    idpreds[key] = predictions[i]

evaluate(idpreds, testset_path, 'svm')
#confusion(idpreds, testset_path, 'svm')


The next three cells are for the randomforest classifier. Two methods, grid search and random search were used to find the optimal parameters for the classifier but take too long to run for it to be feasible to include in the actual classifier. The optimal number of estimators found, 138, was surprisingly consistent for all the test sets.

In [ ]:
#randomforest classifier for the twitter training data
train_vectors, test_vectors = generate_features(training_set, testset, features)
ids = []
for i, key in enumerate(tweetids[training_set]):
    ids.append(key)
randomforest = RandomForestClassifier(n_estimators = 138, max_depth = 50, random_state = 0)
randomforest.fit(train_vectors, tweetgts[training_set])
predictions = randomforest.predict(test_vectors)

#testing on the testsets
idpreds = {key: pred for key, pred in zip(tweetids[testset], predictions)}
testset_path = data_dir + '\\' + testset

evaluate(idpreds, testset_path, 'randomforest')
confusion(idpreds, testset_path, 'randomforest')

In [ ]:
#attempting to optimise the randomforest classifier
#this will take like a year to run 

#data preprocessing
scaler = StandardScaler(with_mean=False)
train_vectors = scaler.fit_transform(train_vectors)
test_vectors = scaler.transform(test_vectors)

#feature selection
selector = SelectKBest(f_classif, k=400)
train_vectors = selector.fit_transform(train_vectors, tweetgts[training_set])
test_vectors = selector.transform(test_vectors)

#hyperparameter tuning
param_grid = {
    'n_estimators': [20, 30, 40],
    'max_depth': [25, 35, 45],
    'max_features':[1,3,5,7],
    'min_samples_leaf':[1,2,3],
    'min_samples_split':[1,2,3]
}

grid = GridSearchCV(RandomForestClassifier(random_state=0), param_grid=param_grid, cv=5, verbose=1)
grid.fit(train_vectors, tweetgts[training_set])

#testing on the testsets
randomforest = grid.best_estimator_
predictions = randomforest.predict(test_vectors)

idpreds = {key: pred for key, pred in zip(tweetids[testset], predictions)}
testset_path = data_dir + '\\' + testset
print(randomforest)
evaluate(idpreds, testset_path, 'randomforest')
confusion(idpreds, testset_path, 'randomforest')

In [ ]:
#attempting to optimise the randomforest classifier using the random search instead of grid search
#this will take like a year to run 

#data preprocessing
scaler = StandardScaler(with_mean=False)
train_vectors = scaler.fit_transform(train_vectors)
test_vectors = scaler.transform(test_vectors)

#feature selection
selector = SelectKBest(f_classif, k='all')
train_vectors = selector.fit_transform(train_vectors, tweetgts[training_set])
test_vectors = selector.transform(test_vectors)

#hyperparameter tuning
param_dist = {"n_estimators": sp_randint(50, 500),
              "max_depth": [50, 60, 70, None]}
random_search = RandomizedSearchCV(RandomForestClassifier(random_state=0), 
                                   param_distributions=param_dist, 
                                   n_iter=10, cv=5, random_state=0, verbose=1)
random_search.fit(train_vectors, tweetgts[training_set])

#testing on the testsets
randomforest = random_search.best_estimator_
predictions = randomforest.predict(test_vectors)

idpreds = {key: pred for key, pred in zip(tweetids[testset], predictions)}
print(randomforest)
evaluate(idpreds, testset_path, 'randomforest')
confusion(idpreds, testset_path, 'randomforest')

The next two cells contain the knn classifier. The second is not used in the main code but was used to find the optimal hyperparameters. It's not included as the actual classifier because running it across enough parameters to be useful takes so long, however it is left here to show the method use to find the parameters in the actual classifier.

In [ ]:
#knn classifier for the twitter training data
train_vectors, test_vectors = generate_features(training_set, testset, features)
knn = KNeighborsClassifier(n_neighbors = 100, weights = 'uniform', algorithm = 'auto', leaf_size = 300, p = 2)
knn.fit(train_vectors, tweetgts[training_set]) 
predictions = knn.predict(test_vectors)
            
#testing on the testsets
idpreds = {key: pred for key, pred in zip(tweetids[testset], predictions)}

print(predictions)
evaluate(idpreds, testset_path, 'knn')
confusion(idpreds, testset_path, 'knn')

In [ ]:
#attempting to optimise the knn classifier
train_vectors, test_vectors = generate_features(training_set, testset, features)
#data preprocessing
scaler = StandardScaler(with_mean=False)
train_vectors = scaler.fit_transform(train_vectors)
test_vectors = scaler.transform(test_vectors)

#feature selection
selector = SelectKBest(chi2, k='all')
train_vectors = selector.fit_transform(train_vectors, tweetgts[training_set])
test_vectors = selector.transform(test_vectors)

#parameter tuning
k_values = [90, 110, 130, 150, 170]
distance_metrics = ['euclidean', 'manhattan']
best_accuracy = 0
best_k = None
best_metric = None
for k in k_values:
    for metric in distance_metrics:
        knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', algorithm='auto', leaf_size=300, p=2, metric=metric)
        knn.fit(train_vectors, tweetgts[training_set]) 
        predictions = knn.predict(test_vectors)
        accuracy = accuracy_score(tweetgts[testset], predictions)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_k = k
            best_metric = metric

#testing on the testsets
knn = KNeighborsClassifier(n_neighbors=best_k, weights='uniform', algorithm='auto', leaf_size=300, p=2, metric=best_metric)
knn.fit(train_vectors, tweetgts[training_set]) 
predictions = knn.predict(test_vectors)

print(best_accuracy, best_k, best_metric)

idpreds = {key: pred for key, pred in zip(tweetids[testset], predictions)}
evaluate(idpreds, testset_path, 'knn')
confusion(idpreds, testset_path, 'knn')

In [ ]:
#building the lstm
#train_vectors, test_vectors = generate_features(training_set, testset, 'embeddings')

#creating the input vector
for i, batch in enumerate(train_vectors):
    input, target_vector = batch
    
    #defining the models
    batch_size = target_vector.shape[0]
    hidden_size = 64
    bidirectional = True
    lstm = torch.nn.LSTM(input_size=input.shape[-1], hidden_size=hidden_size, num_layers=2, batch_first=True, bidirectional=bidirectional)
    output_layer = torch.nn.Linear(hidden_size+hidden_size*bidirectional, 1)
    
    #adding dropout
    dropout_rate = 0.21
    dropout = torch.nn.Dropout(0.21)

    #defining the loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    #defining the optimizer
    optimizer = torch.optim.Adam(lstm.parameters(), lr=0.01)

    #training the model
    epochs = 15
    for epoch in range(epochs):
        target = torch.tensor(target_vector).long()
        target = target.reshape(target.shape[0], 1)
        optimizer.zero_grad()
        output, _ = lstm(input)
        output = dropout(output)
        output = output_layer(output)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

#testing the model
all_test_outputs = []
lstm.eval()
with torch.no_grad():
    for i, batch in enumerate(test_vectors):
        test_input = batch[0]
        test_output, _ = lstm(test_input)
        test_output = dropout(test_output)
        test_output = output_layer(test_output)
        all_test_outputs.append(test_output)

#concatenate test output tensors from all batches into a single tensor
all_test_outputs = torch.cat(all_test_outputs, dim=0)

#make predictions using the concatenated test outputs
predictions = all_test_outputs.argmax(dim=1)

#converting to string output
predictions_list = []
for i, index in enumerate(predictions):
    if index == 0:
        predictions_list.append('positive')
    elif index == 1:
        predictions_list.append('negative')
    elif index == 2:
        predictions_list.append('neutral')

#evaluation
testset_path = data_dir + '\\' + testset
idpreds = {key: pred for key, pred in zip(tweetids[testset], predictions_list)}
evaluate(idpreds, testset_path, 'lstm')
confusion(idpreds, testset_path, 'lstm')

In [57]:
# Build traditional sentiment classifiers. An example classifier name 'svm' is given
# in the code below. You should replace the other two classifier names
# with your own choices. For features used for classifier training, 
# the 'bow' feature is given in the code. But you could also explore the 
# use of other features.
classifier_names = ['svm', 'randomforest', 'knn', 'LSTM']
feature_names = ['bow', 'embeddings']

for classifier in classifier_names:
    for features in feature_names:
        # Skeleton: Creation and training of the classifiers
        if features == 'embeddings' and classifier != 'LSTM':
            continue
        
        if classifier == 'svm':
            # write the svm classifier here
            #svm classifier
            train_vectors, test_vectors = generate_features(training_set, testset, features)
            svm = LinearSVC()
            svm.fit(train_vectors, tweetgts[training_set])
            print('Training ' + classifier)
        elif classifier == 'randomforest':
            # write the classifier 2 here
            #randomforest classifier
            train_vectors, test_vectors = generate_features(training_set, testset, features)
            ids = []
            for i, key in enumerate(tweetids[training_set]):
                ids.append(key)
            randomforest = RandomForestClassifier(n_estimators = 138, max_depth = 50, random_state = 0)
            randomforest.fit(train_vectors, tweetgts[training_set])
            print('Training ' + classifier)
        elif classifier == 'knn':
            # write the classifier 3 here
            #knn classifier
            train_vectors = generate_features(training_set, testset, features)[0]
            knn = KNeighborsClassifier(n_neighbors = 110, weights = 'uniform', algorithm = 'auto', leaf_size = 300, p = 2)
            knn.fit(train_vectors, tweetgts[training_set]) 
            print('Training ' + classifier)
        elif classifier == 'LSTM':
            # write the LSTM classifier here
            if features == 'bow':
                continue
        
            train_vectors, test_vectors = generate_features(training_set, testset, features)
            #creating the input vector
            for i, batch in enumerate(train_vectors):
                input, target_vector = batch
                
                #defining the models
                batch_size = target_vector.shape[0]
                hidden_size = 64
                bidirectional = True
                lstm = torch.nn.LSTM(input_size=input.shape[-1], hidden_size=hidden_size, num_layers=2, batch_first=True, bidirectional=bidirectional)
                output_layer = torch.nn.Linear(hidden_size+hidden_size*bidirectional, 1)
                
                #adding dropout
                dropout_rate = 0.21
                dropout = torch.nn.Dropout(0.21)

                #defining the loss function
                loss_fn = torch.nn.CrossEntropyLoss()

                #defining the optimizer
                optimizer = torch.optim.Adam(lstm.parameters(), lr=0.01)

                #training the model
                epochs = 15
                for epoch in range(epochs):
                    target = torch.tensor(target_vector).long()
                    target = target.reshape(target.shape[0], 1)
                    optimizer.zero_grad()
                    output, _ = lstm(input)
                    output = dropout(output)
                    output = output_layer(output)
                    loss = loss_fn(output, target)
                    loss.backward()
                    optimizer.step()
            print('Training ' + classifier)
        else:
            print('Unknown classifier name' + classifier)
            continue

        # Prediction performance of the classifiers
        for testset in testsets:
            test_vectors = generate_features(training_set, testset, features)[1]
            if classifier == 'svm':
                predictions = svm.predict(test_vectors)
            elif classifier == 'randomforest':
                predictions = randomforest.predict(test_vectors)
            elif classifier == 'knn':
                predictions = knn.predict(test_vectors)
            elif classifier == 'LSTM':
                all_test_outputs = []
                lstm.eval()
                with torch.no_grad():
                    for i, batch in enumerate(test_vectors):
                        test_input = batch[0]
                        test_output, _ = lstm(test_input)
                        test_output = dropout(test_output)
                        test_output = output_layer(test_output)
                        all_test_outputs.append(test_output)

                #concatenate test output tensors from all batches into a single tensor
                all_test_outputs = torch.cat(all_test_outputs, dim=0)

                #make predictions 
                prediction_tensor = all_test_outputs.argmax(dim=1)

                #converting to string output
                predictions = []
                for i, index in enumerate(prediction_tensor):
                    if index == 0:
                        predictions.append('positive')
                    elif index == 1:
                        predictions.append('negative')
                    elif index == 2:
                        predictions.append('neutral')
            id_preds = {key: pred for key, pred in zip(tweetids[testset], predictions)}
            # write the prediction and evaluation code here

            testset_name = testset
            testset_path = join(data_dir, testset_name)
            evaluate(id_preds, testset_path, features + '-' + classifier)

Training svm
d:\Documents\CS_MSc\2-Natural-Language-Processing\Coursework\Data\twitter-test1.txt (bow-svm): 0.552
d:\Documents\CS_MSc\2-Natural-Language-Processing\Coursework\Data\twitter-test2.txt (bow-svm): 0.578
d:\Documents\CS_MSc\2-Natural-Language-Processing\Coursework\Data\twitter-test3.txt (bow-svm): 0.535
Training randomforest
d:\Documents\CS_MSc\2-Natural-Language-Processing\Coursework\Data\twitter-test1.txt (bow-randomforest): 0.360
d:\Documents\CS_MSc\2-Natural-Language-Processing\Coursework\Data\twitter-test2.txt (bow-randomforest): 0.393
d:\Documents\CS_MSc\2-Natural-Language-Processing\Coursework\Data\twitter-test3.txt (bow-randomforest): 0.336
Training knn
d:\Documents\CS_MSc\2-Natural-Language-Processing\Coursework\Data\twitter-test1.txt (bow-knn): 0.319
d:\Documents\CS_MSc\2-Natural-Language-Processing\Coursework\Data\twitter-test2.txt (bow-knn): 0.351
d:\Documents\CS_MSc\2-Natural-Language-Processing\Coursework\Data\twitter-test3.txt (bow-knn): 0.296


C:\Users\benla\AppData\Local\Temp\ipykernel_14676\4159776461.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target_vector).long()


Training LSTM
d:\Documents\CS_MSc\2-Natural-Language-Processing\Coursework\Data\twitter-test1.txt (embeddings-LSTM): 0.318
d:\Documents\CS_MSc\2-Natural-Language-Processing\Coursework\Data\twitter-test2.txt (embeddings-LSTM): 0.311
d:\Documents\CS_MSc\2-Natural-Language-Processing\Coursework\Data\twitter-test3.txt (embeddings-LSTM): 0.330
